<a href="https://colab.research.google.com/github/gh-ita/Spam-Detectiion/blob/main/DetectionSpam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords,wordnet
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Prétraitement

In [ ]:
texte = open('/content/drive/MyDrive/SMSSpamCollection.txt',"r")


## Séparation des spam et des ham

In [ ]:
ham = []
spam = []
for line in texte :
  if line[0] == 'h':
    ham.append(line[4:])
  else :
    spam.append(line[5:])



In [ ]:
print(ham)

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'Ok lar... Joking wif u oni...\n', 'U dun say so early hor... U c already then say...\n', "Nah I don't think he goes to usf, he lives around here though\n", 'Even my brother is not like to speak with me. They treat me like aids patent.\n', "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune\n", "I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.\n", "I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times.\n", 'I HAVE A DATE ON SUNDAY WITH WILL!!\n', "Oh k...i'm watching here:)\n", 'Eh u remember how 2 spell his name... Yes i did. He v naughty make until i v wet.\n',

In [ ]:
def netoyage(corpus):
  stopwords_en = stopwords.words('english')
  for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()
    for p in string.punctuation :
      corpus[i] = corpus[i].replace(p," ")
    liste = corpus[i].split()
    for mot in liste :
      if mot in stopwords_en :
        liste.remove(mot)
    corpus[i] = " ".join(liste)
  return corpus

In [ ]:
ham = netoyage(ham)

In [ ]:
print(ham)

['go jurong point crazy available in bugis n great world la e buffet cine got amore wat', 'ok lar joking wif u oni', 'u dun say early hor u c already say', 'nah don think goes usf lives around though', 'even brother not like speak treat me like aids patent', 'per request melle melle oru minnaminunginte nurungu vettam been set callertune all callers press 9 copy your friends callertune', 'm gonna home soon t want talk this stuff anymore tonight k i ve cried enough today', 've searching the right words thank breather promise wont take help for granted will fulfil promise have wonderful a blessing all times', 'have date sunday will', 'oh k m watching', 'eh u remember 2 spell name yes did v naughty make i v wet', 'fine way u feel that s the way gota b', 'that seriously you spell name', 'i‘m going try 2 months ha ha joking', 'ü pay first lar when da stock comin', 'aft finish lunch i go str lor ard 3 smth lor u finish ur lunch already', 'ffffffffff alright way can meet with sooner', 'forced 

In [ ]:
spam = netoyage(spam)

In [ ]:
len(ham)

4827

In [ ]:
len(spam)

748

In [ ]:
from collections import Counter
occurance = Counter(ham)
print(occurance)

Counter({'sorry ll call later': 30, 'ok': 20, 'cant pick phone right pls send message': 12, 'okie': 7, 'ok lor': 5, 'sorry ll call later meeting': 4, 'wen ur lovable bcums angry wid u dnt take seriously coz angry d childish n true way showing deep affection care n luv kettoda manda nice day da': 4, 'opinion me 1 2 jada 3 kusruthi 4 lovable 5 silent 6 spl character 7 matured 8 stylish 9 simple pls reply': 4, '7 wonders my world 7th 6th ur style 5th ur smile 4th ur personality 3rd ur nature 2nd ur sms 1st ur lovely friendship good morning dear': 4, 'say slowly god love amp need clean heart your blood send to ten special people amp u c miracle tomorrow it pls pls it': 4, 'per request melle melle oru minnaminunginte nurungu vettam been set callertune all callers press 9 copy your friends callertune': 3, 'calls messages missed calls': 3, 'will in place that man': 3, 'entered cabin pa said happy b day boss i felt special askd 4 lunch lunch invited to apartment went': 3, 'no will check rooms 

In [ ]:
len(occurance.values())

4497

In [ ]:
spamOccurance = Counter(spam)
len(spamOccurance)

630

In [ ]:
hamWithoutOccurance = np.array(list(occurance.keys()))
spamWithoutOccurance = np.array(list(spamOccurance.keys()))

In [ ]:
print(len(hamWithoutOccurance[0].split()))

17


In [ ]:
print(len(hamWithoutOccurance))

4497


Aprés avoir nettoyer les données et enlever les occurances on constate que la taille des données ham est de 4497 et celle des données spam est de 630. Ceci montre un fort déséquilibre au niveau des données.



# Data augmentation for spam data

In [ ]:
def synonym_replacement(sentence):
    synonyms = []
    for word in sentence.split():
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
    if synonyms:
        return ' '.join(synonyms)
    else:
        return sentence

spam_data_augmented = [synonym_replacement(sentence) for sentence in spamWithoutOccurance]


Génération des spam à partir des spam originaux, en remplaçant les termes par leurs synonymes.

# Entrainement du modéle

In [ ]:

#Convert the data into a numerical format
def TF(term,sentence):
  #calculate tf for a term
  occ = sentence.count(term)
  nterms = len(sentence.split())
  return occ/nterms

def IDF(terme,data):
  #calculate idf for a term
  ndoc = 0
  for i in data :
    if terme in i :
      ndoc += 1
  return np.log(len(data)/ndoc)

def TF_IDF(terme,idf,sentence):
  tf = TF(terme,sentence)
  return tf*idf

def dictionnaire(data):
  text = ' '.join(data)
  words = text.split()
  occ = Counter(words)
  uniWords = np.array(list(occ.keys()))
  return uniWords


In [ ]:
def matrice_sparse(data):
  uniWords = dictionnaire(data)
  Matrix = np.zeros([len(data),len(uniWords)])
  print(len(data),len(uniWords))
  for i in uniWords :
    idf = IDF(i,data)
    wordInd = np.where(uniWords == i)
    for j in data :
      tf_idf = TF_IDF(i,idf,j)
      sentIndex = np.where(data == j)
      Matrix[sentIndex[0],wordInd[0]] = tf_idf
  return Matrix


In [ ]:
matrice = matrice_sparse(spamWithoutOccurance)

630 2901


In [ ]:
(n,p) = matrice.shape

for i in range(n):
  for j in range(p) :
    matrice[i,j] = round(matrice[i,j],2)


In [ ]:
hamClass = []
for i in range(len(hamWithoutOccurance)):
  hamClass.append('ham')
for i in range(len(spamWithoutOccurance)):
  hamClass.append('spam')

In [ ]:
column_data = {
    "classe" :hamClass,
    "message":np.concatenate((hamWithoutOccurance, spamWithoutOccurance))
}

In [ ]:
dataframe = pd.DataFrame(column_data)

In [ ]:
dataframe.head()

,classe,message
0,ham,go jurong point crazy available in bugis n gre...
1,ham,ok lar joking wif u oni
2,ham,u dun say early hor u c already say
3,ham,nah don think goes usf lives around though
4,ham,even brother not like speak treat me like aids...


In [ ]:
spamTrain, spamsTest = train_test_split(dataframe,train_size=0.7,random_state = 1)
